In [12]:
from selenium import webdriver
import time
import os
import pandas as pd
import random
from dotenv import load_dotenv

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [13]:
url2 = "https://www.indeed.com/jobs?q=software+engineer+intern&l=United+States"

In [14]:
# sets up the chrome driver and opens the webpage
load_dotenv()
service = Service(os.getenv("CHROMEDRIVER_PATH"))
driver = webdriver.Chrome(service=service)
driver.get(url2)
driver.implicitly_wait(10)

In [15]:
companyNames = []
titles = []
jobLocations = []
links = []
datePosted = []

In [16]:
while True:
    try:
        jobElements = driver.find_elements(By.CLASS_NAME, "cardOutline")
        time.sleep(3)
        
        for job in jobElements:
            companyName = job.find_element(By.CSS_SELECTOR, '[data-testid="company-name"]').text
            companyNames.append(companyName)
            title = job.find_element(By.CLASS_NAME, "jobTitle").text
            titles.append(title)
            jobLocation = job.find_element(By.CSS_SELECTOR, '[data-testid="text-location"]').text
            jobLocations.append(jobLocation)
            link = job.find_element(By.CLASS_NAME, "jobTitle").find_element(By.TAG_NAME, 'a').get_attribute('href')
            links.append(link)
            date = job.find_element(By.CSS_SELECTOR, '[data-testid="myJobsStateDate"]').text[6:].strip()
            datePosted.append(date)
        
        nextPage = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Next Page']"))
        )
        
        nextPage.click()
        time.sleep(random.randint(3, 6))
    
    except (NoSuchElementException, TimeoutException):
        break
    
    except Exception as e:
        time.sleep(random.randint(2, 5))


In [17]:
companyNameData = pd.DataFrame(companyNames, columns=["Company Name"])
titleData = pd.DataFrame(titles, columns=["Title"])
jobLocationData = pd.DataFrame(jobLocations, columns=["Job Location"])
linkData = pd.DataFrame(links, columns=["Links"])
dateData = pd.DataFrame(datePosted, columns=["Date Posted"])

In [18]:
jobData = pd.DataFrame()
jobData = pd.concat([companyNameData, titleData, jobLocationData, linkData, dateData], axis=1)
numColumns = (jobData.shape[1]*2)-1

for i in range(1, numColumns, 2):
    jobData.insert(i, f'Empty_{i}', '')
    
jobData.index = jobData.index+1

In [19]:
jobData

,Company Name,Empty_1,Title,Empty_3,Job Location,Empty_5,Links,Empty_7,Date Posted
1,SLPT Global Pump Group,,Software Engineer,,"Warren, MI 48089",,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,,er\nActive 2 days ago
2,Kayhan Space,,Junior Backend Software Engineer,,"Broomfield, CO",,https://www.indeed.com/rc/clk?jk=bba02e4d51ba1...,,NaN


In [20]:
len(jobData)

2

In [387]:
jobData.to_csv("Indeed_SWE_Intern_Jobs.csv")

In [388]:
driver.quit()